In [4]:
from src.binaryClassificationModel import BinaryClassificationModel
import pickle
import torch

from src.evaluation import Evaluation

with open('../data/processed/preprocessed.pkl', 'rb') as f:
    params = pickle.load(f)

In [ ]:
model = BinaryClassificationModel(params['X_train'].shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [3]:
from src.binaryClassificationDataset import BinaryClassificationDataset
from torch.utils.data import DataLoader

dataset = BinaryClassificationDataset(list(zip(params['X_train'], params['y_train'])))
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

for epoch in range(30):
    model.train()
    total_loss = 0

    for X_batch, y_batch in dataloader:
        optimizer.zero_grad()
        y_batch = y_batch.unsqueeze(1)

        logits = model(X_batch)
        loss = model.loss(logits, y_batch)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")


Epoch 1, Loss: 0.4979
Epoch 2, Loss: 0.2652
Epoch 3, Loss: 0.1457
Epoch 4, Loss: 0.1015
Epoch 5, Loss: 0.0847
Epoch 6, Loss: 0.0763
Epoch 7, Loss: 0.0708
Epoch 8, Loss: 0.0671
Epoch 9, Loss: 0.0646
Epoch 10, Loss: 0.0622
Epoch 11, Loss: 0.0605
Epoch 12, Loss: 0.0591
Epoch 13, Loss: 0.0576
Epoch 14, Loss: 0.0563
Epoch 15, Loss: 0.0551
Epoch 16, Loss: 0.0543
Epoch 17, Loss: 0.0532
Epoch 18, Loss: 0.0520
Epoch 19, Loss: 0.0511
Epoch 20, Loss: 0.0508
Epoch 21, Loss: 0.0500
Epoch 22, Loss: 0.0494
Epoch 23, Loss: 0.0484
Epoch 24, Loss: 0.0479
Epoch 25, Loss: 0.0472
Epoch 26, Loss: 0.0467
Epoch 27, Loss: 0.0463
Epoch 28, Loss: 0.0459
Epoch 29, Loss: 0.0452
Epoch 30, Loss: 0.0452


In [4]:
test_set = BinaryClassificationDataset(list(zip(params['X_test'], params['y_test'])))
test_loader = DataLoader(test_set, batch_size=10, shuffle=True)

model.eval()
test_loss = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_batch = y_batch.unsqueeze(1)
        logits = model(X_batch)
        loss = model.loss(logits, y_batch)
        test_loss += loss.item()

print(f"Test Loss: {test_loss/len(test_loader):.4f}")

Test Loss: 0.0740


In [5]:
torch.save(model.state_dict(), "../models/BinaryClassificationModel.pth")

In [5]:
evaluation = Evaluation(params['X_test'],params['y_test'])
evaluation()

Model loaded successfully.
Evaluation Metrics: {'accuracy': 0.9829, 'precision': 0.9846938775510204, 'recall': 0.9841558914587507, 'f1_score': 0.9844248110028236}
